In [22]:
#!/usr/bin/python
import cv2
import numpy as np
import random   

#print the triangle coordinate
def save_coordinate2txt(img,subdiv,points,coordinates) :
    feature_num=78   #此处修改特征点数量
    size = img.shape
    r = (0,0,size[1], size[0]);
    landmarks=np.array(points);
    triangleList=subdiv.getTriangleList();
    for t in triangleList :
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        for num1 in range(0,feature_num):
            f1=f2=f3=100;
            if t[0]==landmarks[num1,0] and t[1]==landmarks[num1,1]:
                f1=num1
                for num2 in range(0,feature_num):
                    if pt2[0]==landmarks[num2,0] and pt2[1]==landmarks[num2,1]:
                        f2=num2
                        for num3 in range(0,feature_num):
                            if pt3[0] == landmarks[num3,0] and pt3[1]==landmarks[num3,1]:
                                f3=num3
                                coordinates.append((int(f1), int(f2),int(f3)))
    
    text=np.array(coordinates)
    np.savetxt("/home/fengchao/桌面/humantri.txt",text,fmt='%i %i %i')

In [23]:
# Check if a point is inside a rectangle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Draw a point
def draw_point(img, p, color ) :
    cv2.circle( img, p, 2, color, -1, 8, 0 )


# Draw delaunay triangles
def draw_delaunay(img, subdiv, delaunay_color ) :

    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList :
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
        
            cv2.line(img, pt1, pt2, delaunay_color, 1, 8, 0)
            cv2.line(img, pt2, pt3, delaunay_color, 1, 8, 0)
            cv2.line(img, pt3, pt1, delaunay_color, 1, 8, 0)

In [25]:
if __name__ == '__main__':

    # Define window names
    win_delaunay = "Delaunay Triangulation"
    win_voronoi = "Voronoi Diagram"

    # Turn on animation while drawing triangles
    animate = False
    
    # Define colors for drawing.
    delaunay_color = (255,255,255)
    points_color = (0, 0, 255)
    
    # Read in the image.
    img = cv2.imread("/home/fengchao/桌面/facedata/78points/h20.tif");#路径修改成画三角的图片
    
    # Keep a copy around
    img_orig = img.copy();
    
    # Rectangle to be used with Subdiv2D
    size = img.shape
    rect = (0, 0, size[1], size[0])
    
    # Create an instance of Subdiv2D
    subdiv = cv2.Subdiv2D(rect);
    
    # Create an array of points.
    points = [];
    
    # Create an array of coordinates.
    coordinates= [];
    
    # Read in the points from a text file
    with open("/home/fengchao/桌面/facedata/78points/h20.txt") as file :#图片对应的landmarks的坐标，每组坐标点都以行来区分注意源文件换行
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    
    # Insert points into subdiv
    for p in points :
        subdiv.insert(p)        
            # Show animation
        if animate :
            img_copy = img_orig.copy()
            # Draw delaunay triangles
            draw_delaunay( img_copy, subdiv, (255, 255, 255) );
            cv2.imshow(win_delaunay, img_copy)
            cv2.waitKey(100)
    # Draw delaunay triangles
    draw_delaunay( img, subdiv, (255, 255, 255) );

    # Draw points
    for p in points :draw_point(img, p, (0,0,255))
        
    #print the coordinates
    save_coordinate2txt(img,subdiv,points,coordinates);